In [3]:
# --------------------------------------------------
# Cell 1: 导入库并设置项目结构
# --------------------------------------------------
import os
import re
import sys
import warnings
from datetime import datetime
from pathlib import Path
from time import perf_counter

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

base_path = Path.cwd()
report_path = base_path / "报告数据"
# 输入路径
input_path = report_path / "输入"
anjian_data_path = input_path / "安监数据"
base_data_path = input_path / "basic_data.xlsx"
# 输出路径
output_path = report_path / "输出"
# 中间过程文件路径（自动创建，用于存放临时文件）
temp_path = report_path / "temp"
upload_split_path = temp_path / "1_待上传猪猪云文件"
zhuzhuyun_download_path = temp_path / "2_猪猪云下载数据"
zhuzhuyun_merge_path = temp_path / "3_猪猪云合并数据"
pycharm_input_path = temp_path / "4_logistics数据"
transit_data_path = temp_path / "5_中转数据"

In [4]:
# --- 1. 报告周期配置（每月仅修改此处） ---
CURRENT_YEAR_MONTH = "202506"
PREVIOUS_YEAR_MONTH = "202505"
LAST_YEAR_MONTH = "202406"
CURRENT_MONTH_DISPLAY = f"{int(CURRENT_YEAR_MONTH[-2:])}月"  # 无需修改

In [ ]:
# ==============================================================================
# Cell 2: 中通报告所需表格生成
# ==============================================================================

# --- 1. 路径和常量定义 ---
ROOT_PATH = Path.cwd()
OUTPUT_DIR = ROOT_PATH / "报告数据" / "输出" / "2_中通报告表格"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

BASIC_DATA_FILE = ROOT_PATH / "报告数据" / "输入" / "basic_data.xlsx"


def get_file_paths(period_label):
    base_dir_map = {
        "本期": ROOT_PATH / "报告数据" / "输出" / "data_analysis_result",
        "上月": ROOT_PATH
        / "报告数据"
        / "输入"
        / "historical_data"
        / f"{PREVIOUS_YEAR_MONTH}"
        / f"data_analysis_result_{PREVIOUS_YEAR_MONTH}",
        "去年同期": ROOT_PATH
        / "报告数据"
        / "输入"
        / "historical_data"
        / f"{LAST_YEAR_MONTH}"
        / f"data_analysis_result_{LAST_YEAR_MONTH}",
    }
    path = base_dir_map.get(period_label)
    if period_label != "本期" and (not path or not path.exists()):
        month_var = PREVIOUS_YEAR_MONTH if period_label == "上月" else LAST_YEAR_MONTH
        print(f"  - ⚠️ 警告: 路径 {path} 不存在，尝试回退到旧的'分析总报告'文件...")
        return {
            "report": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / month_var
            / f"分析总报告_{month_var}.xlsx"
        }
    return {"data_analysis": path} if path else {}


# 明确定义各个公司列表的用途
COMPANIES_ALL = [
    "中通",
    "圆通",
    "极兔",
    "申通",
    "韵达",
    "顺丰",
    "京东",
    "EMS",
    "德邦",
    "快包",
]
# 用于行业对比的公司列表，明确排除'快包'
COMPANIES_FOR_INDUSTRY_COMPARISON = [c for c in COMPANIES_ALL if c != "快包"]
COMPANY_FILE_MAP = {
    "中通": "中通",
    "圆通": "圆通",
    "极兔": "极兔",
    "申通": "申通",
    "韵达": "韵达",
    "顺丰": "顺丰",
    "京东": "京东",
    "EMS": "EMS",
    "德邦": "德邦",
    "快包": "邮政",
}
COMPANIES_TONGDATU = ["圆通", "申通", "中通", "极兔", "韵达"]
METRICS_TABLE1 = {
    "全程时限（小时）": "全程时限",
    "72小时妥投率": "72小时准时率",
    "48小时妥投率": "48小时准时率",
}
METRICS_TABLE2 = {
    "寄出地处理时限": "寄出地处理时限",
    "运输时限": "运输时限",
    "寄达地处理时限": "寄达地处理时限",
    "投递时限": "投递时限",
}
ALL_METRICS = {**METRICS_TABLE1, **METRICS_TABLE2}
HIGHER_IS_BETTER = ["72小时妥投率", "48小时妥投率"]


def load_top_cities(file_path: Path) -> set:
    sheet_name, column_name = "30_top_volume_city_2024", "城市"
    print(f"\n正在从 '{file_path.name}' 加载 Top 30 城市列表...")
    try:
        if not file_path.exists():
            raise FileNotFoundError(f"基础数据文件不存在: {file_path}")
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        if column_name not in df.columns:
            raise ValueError(
                f"在Sheet '{sheet_name}' 中未找到名为 '{column_name}' 的列。"
            )
        cities_set = set(df[column_name].dropna().astype(str).str.strip().tolist())
        if not cities_set:
            print(f"警告: 从 '{file_path.name}' 中加载的城市列表为空。")
        else:
            print(f"成功加载 {len(cities_set)} 个 Top 30 城市。")
        return cities_set
    except Exception as e:
        print(f"错误: 加载Top 30城市列表失败！错误信息: {e}")
        sys.exit(1)


# ==============================================================================
# Part 1: 数据计算与提取引擎
# ==============================================================================
def calculate_metrics_from_result_files(data_analysis_dir, top_30_cities):
    df_result = pd.DataFrame()
    if not data_analysis_dir or not data_analysis_dir.exists():
        print(f"  - ⚠️ 警告：数据分析结果目录不存在: {data_analysis_dir}")
        return df_result
    for company_key, file_prefix in COMPANY_FILE_MAP.items():
        file_path = data_analysis_dir / f"{file_prefix}_data_analysis_result.xlsx"
        if not file_path.exists():
            continue
        try:
            df = pd.read_excel(file_path, sheet_name="线路详细数据")
            if df.empty:
                continue
        except Exception as e:
            print(f"  - ⚠️ 警告：读取 {file_path.name} 的 '线路详细数据' sheet失败: {e}")
            continue

        # 计算除48h妥投率外的所有指标
        df["72小时准时率"] = df["全程时限"] <= 72
        for report_metric, source_metric in ALL_METRICS.items():
            if report_metric == "48小时妥投率":
                continue
            if source_metric in df.columns:
                df_result.loc[report_metric, company_key] = df[source_metric].mean()

        # 计算48小时妥投率 (基于Top 30城市互寄数据)
        top_30_data = df[
            (df["寄出城市"].isin(top_30_cities)) & (df["寄达城市"].isin(top_30_cities))
        ]
        rate_48h = (
            (top_30_data["全程时限"] <= 48).mean() if not top_30_data.empty else 0.0
        )
        df_result.loc["48小时妥投率", company_key] = rate_48h
    return df_result


def get_period_data(period_label, top_30_cities):
    print(f"\n--- 正在处理 {period_label} 数据 ---")
    paths = get_file_paths(period_label)
    data_analysis_dir = paths.get("data_analysis")
    df_final = calculate_metrics_from_result_files(data_analysis_dir, top_30_cities)

    if df_final.empty:
        # 兼容旧逻辑
        report_file_path = paths.get("report")
        if report_file_path and report_file_path.exists():
            print(
                f"  - ⚠️ 警告：在 {data_analysis_dir} 未找到数据, 尝试从旧报告 {report_file_path.name} 读取..."
            )
        else:
            print(f"  - ⚠️ 警告：{period_label} 未能计算出任何指标，数据源缺失。")
        return pd.DataFrame()

    # 使用不含'快包'的列表进行行业对比
    industry_competitors = df_final.columns.intersection(
        COMPANIES_FOR_INDUSTRY_COMPARISON
    ).tolist()
    if industry_competitors:
        df_final["行业均值"] = df_final[industry_competitors].mean(axis=1)
        for metric, row in df_final.iterrows():
            if metric in HIGHER_IS_BETTER:
                df_final.loc[metric, "行业最优"] = row[industry_competitors].max()
            else:
                df_final.loc[metric, "行业最优"] = row[industry_competitors].min()

    # 计算通达兔指标
    existing_tongdatu = df_final.columns.intersection(COMPANIES_TONGDATU).tolist()
    if existing_tongdatu:
        df_final["通达兔均值"] = df_final[existing_tongdatu].mean(axis=1)
        for metric, row in df_final.iterrows():
            if metric in HIGHER_IS_BETTER:
                df_final.loc[metric, "通达兔最优"] = row[existing_tongdatu].max()
            else:
                df_final.loc[metric, "通达兔最优"] = row[existing_tongdatu].min()

    print(f"  ✅ {period_label} 数据提取完成。")
    return df_final


# ==============================================================================
# Part 2 & 3: 报告生成与写入函数
# ==============================================================================
def create_report_table(df_current, df_mom, df_yoy, metrics_dict):
    report_cols = [
        "指标",
        "类别",
        "中通",
        "通达兔均值",
        "通达兔最优",
        "行业均值",
        "行业最优",
        "排名",
    ]
    value_cols = ["中通", "通达兔均值", "通达兔最优", "行业均值", "行业最优"]
    if not df_mom.empty:
        df_mom = df_mom.reindex(columns=df_current.columns)
    if not df_yoy.empty:
        df_yoy = df_yoy.reindex(columns=df_current.columns)
    report_rows = [
        f"{metric}_{p}" for metric in metrics_dict for p in ["本期", "环比", "同比"]
    ]
    table_df = pd.DataFrame(index=report_rows, columns=report_cols).fillna("")

    for metric in metrics_dict.keys():
        table_df.loc[f"{metric}_本期", "指标"] = metric.replace("（小时）", "")
        table_df.loc[f"{metric}_本期", "类别"] = CURRENT_MONTH_DISPLAY
        table_df.loc[f"{metric}_环比", "类别"] = "环比"
        table_df.loc[f"{metric}_同比", "类别"] = "同比"
        if not df_current.empty and metric in df_current.index:
            table_df.loc[f"{metric}_本期", value_cols] = df_current.loc[
                metric, value_cols
            ].values
        if not df_mom.empty and metric in df_mom.index:
            delta = df_current.loc[metric, value_cols] - df_mom.loc[metric, value_cols]
            table_df.loc[f"{metric}_环比", value_cols] = delta.round(4).values
        if not df_yoy.empty and metric in df_yoy.index:
            delta = df_current.loc[metric, value_cols] - df_yoy.loc[metric, value_cols]
            table_df.loc[f"{metric}_同比", value_cols] = delta.round(4).values

        ascending = metric not in HIGHER_IS_BETTER
        # (修改) 排名计算排除'快包'
        if not df_current.empty and metric in df_current.index:
            current_ranks = df_current.loc[
                metric, COMPANIES_FOR_INDUSTRY_COMPARISON
            ].rank(method="min", ascending=ascending, na_option="bottom")
            table_df.loc[f"{metric}_本期", "排名"] = current_ranks.get("中通")
        if not df_mom.empty and metric in df_mom.index:
            mom_ranks = df_mom.loc[metric, COMPANIES_FOR_INDUSTRY_COMPARISON].rank(
                method="min", ascending=ascending, na_option="bottom"
            )
            table_df.loc[f"{metric}_环比", "排名"] = mom_ranks.get("中通")
        if not df_yoy.empty and metric in df_yoy.index:
            yoy_ranks = df_yoy.loc[metric, COMPANIES_FOR_INDUSTRY_COMPARISON].rank(
                method="min", ascending=ascending, na_option="bottom"
            )
            table_df.loc[f"{metric}_同比", "排名"] = yoy_ranks.get("中通")
    return table_df.reset_index(drop=True)


def write_formatted_excel(writer, df, sheet_name, table_metrics_dict):
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    float_format = workbook.add_format({"num_format": "0.00"})
    percent_format = workbook.add_format({"num_format": "0.00%"})
    integer_format = workbook.add_format({"num_format": "0"})
    rate_metrics_names = [metric for metric in table_metrics_dict if "率" in metric]
    for row_num in range(len(df)):
        original_metric_name = list(table_metrics_dict.keys())[row_num // 3]
        for col_num in range(2, 7):
            cell_value = df.iloc[row_num, col_num]
            if pd.isna(cell_value) or cell_value == "":
                continue
            if original_metric_name in rate_metrics_names:
                worksheet.write_number(row_num + 1, col_num, cell_value, percent_format)
            else:
                worksheet.write_number(row_num + 1, col_num, cell_value, float_format)
        rank_value = df.iloc[row_num, 7]
        if pd.notna(rank_value) and isinstance(rank_value, (int, float)):
            worksheet.write_number(row_num + 1, 7, rank_value, integer_format)
    worksheet.set_column("A:A", 18)
    worksheet.set_column("B:B", 8)
    worksheet.set_column("C:H", 12)


# ==============================================================================
# 最终执行入口
# ==============================================================================
if __name__ == "__main__":
    # 假设这些变量已在别处定义 (你需要确保它们存在)
    # CURRENT_YEAR_MONTH, PREVIOUS_YEAR_MONTH, LAST_YEAR_MONTH, CURRENT_MONTH_DISPLAY

    print(f"🚀 开始生成 {CURRENT_YEAR_MONTH} 月度中通报告...")
    top_30_cities = load_top_cities(BASIC_DATA_FILE)
    df_current = get_period_data("本期", top_30_cities)
    df_mom = get_period_data("上月", top_30_cities)
    df_yoy = get_period_data("去年同期", top_30_cities)

    if df_current.empty:
        print("🛑 本期数据未能成功提取，无法生成报告。")
        sys.exit(1)

    table1_df = create_report_table(df_current, df_mom, df_yoy, METRICS_TABLE1)
    table2_df = create_report_table(df_current, df_mom, df_yoy, METRICS_TABLE2)

    output_path = OUTPUT_DIR / f"中通报告数据_{CURRENT_YEAR_MONTH}.xlsx"
    with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
        write_formatted_excel(writer, table1_df, "全程时限及排名", METRICS_TABLE1)
        write_formatted_excel(writer, table2_df, "分环节时限及排名", METRICS_TABLE2)

    print(f"\n🎉 最终报告已成功保存至: {output_path}")

🚀 开始生成 202506 月度中通报告...

正在从 'basic_data.xlsx' 加载 Top 30 城市列表...
成功加载 30 个 Top 30 城市。

--- 正在处理 本期 数据 ---
  ✅ 本期 数据提取完成。

--- 正在处理 上月 数据 ---
  ✅ 上月 数据提取完成。

--- 正在处理 去年同期 数据 ---
  ✅ 去年同期 数据提取完成。

🎉 最终报告已成功保存至: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输出/2_中通报告表格/中通报告数据_202506.xlsx


In [6]:
# ==============================================================================
# Cell 3.1: 邮政报告所需表格生成 表1 全程时限、72小时妥投率及中转次数排名
# ==============================================================================
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# --- 1. 路径和常量定义 ---
ROOT_PATH = Path.cwd()
POSTAL_OUTPUT_DIR = ROOT_PATH / "报告数据" / "输出" / "3_邮政报告表格"
POSTAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def get_file_paths(period_label):
    """获取指定时期的所有文件路径"""
    if period_label == "本期":
        return {
            "data_analysis": ROOT_PATH / "报告数据" / "输出" / "data_analysis_result",
            "turnover": ROOT_PATH / "报告数据" / "temp" / "5_中转数据",
        }
    elif period_label == "上月":
        return {
            "data_analysis": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / f"{PREVIOUS_YEAR_MONTH}"
            / f"data_analysis_result_{PREVIOUS_YEAR_MONTH}",
            "turnover": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / f"{PREVIOUS_YEAR_MONTH}"
            / f"中转数据_{PREVIOUS_YEAR_MONTH}",
        }
    elif period_label == "去年同期":
        return {
            "data_analysis": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / f"{LAST_YEAR_MONTH}"
            / f"data_analysis_result_{LAST_YEAR_MONTH}",
            "turnover": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / f"{LAST_YEAR_MONTH}"
            / f"中转数据_{LAST_YEAR_MONTH}",
        }
    return {}


# 明确定义各个公司列表的用途
COMPANIES_ALL = [
    "中通",
    "圆通",
    "极兔",
    "申通",
    "韵达",
    "顺丰",
    "京东",
    "EMS",
    "德邦",
    "快包",
]
# 用于行业对比的公司列表，明确排除'快包'
COMPANIES_FOR_INDUSTRY_COMPARISON = [c for c in COMPANIES_ALL if c != "快包"]
COMPANY_FILE_MAP = {
    "EMS": "EMS",
    "德邦": "德邦",
    "极兔": "极兔",
    "圆通": "圆通",
    "顺丰": "顺丰",
    "中通": "中通",
    "京东": "京东",
    "韵达": "韵达",
    "申通": "申通",
    "快包": "邮政",
}
POSTAL_METRICS_MAP = {
    "全程时限（小时）": "全程时限",
    "72小时妥投率": "72小时准时率",
    "48小时妥投率": "48小时准时率",
    "中转次数": "平均中转次数",
}
HIGHER_IS_BETTER = ["72小时妥投率", "48小时妥投率"]


# ==============================================================================
# Part 1: 数据提取函数
# ==============================================================================
def get_turnover_data(turnover_dir):
    """专门从中转数据文件夹提取中转次数"""
    if not turnover_dir or not turnover_dir.exists():
        print(f"  - ⚠️ 警告：中转数据目录不存在: {turnover_dir}")
        return pd.DataFrame()
    turnover_results = {}
    files = list(turnover_dir.glob("*.xlsx"))
    for company_key, file_prefix in COMPANY_FILE_MAP.items():
        if company_key == "快包":
            company_file = next(
                (f for f in files if "快包" in f.name and "异常" not in f.name), None
            )
            if not company_file:
                company_file = next(
                    (f for f in files if "邮政" in f.name and "异常" not in f.name),
                    None,
                )
        else:
            company_file = next(
                (f for f in files if file_prefix in f.name and "异常" not in f.name),
                None,
            )
        if company_file:
            try:
                df = pd.read_excel(company_file)
                if "平均中转次数" in df.columns:
                    turnover_results[company_key] = df["平均中转次数"].mean(skipna=True)
            except Exception as e:
                print(f"  - ⚠️ 警告：读取中转文件 {company_file.name} 失败: {e}")
    if not turnover_results:
        print(f"  - ⚠️ 警告：在 {turnover_dir} 中未能提取到任何中转数据。")
        return pd.DataFrame()
    df_turnover = pd.Series(turnover_results).to_frame(name="中转次数").T

    # 使用不含'快包'的列表进行行业对比
    industry_competitors = [
        c for c in df_turnover.columns if c in COMPANIES_FOR_INDUSTRY_COMPARISON
    ]
    if industry_competitors:
        df_turnover["行业均值"] = df_turnover[industry_competitors].mean(axis=1)
        df_turnover["行业最优"] = df_turnover[industry_competitors].min(axis=1)
    return df_turnover


def get_postal_period_data(period_label):
    """为邮政报告提取所有数据，整合了两个数据源"""
    print(f"\n--- 正在处理邮政报告的 {period_label} 数据 ---")
    paths = get_file_paths(period_label)
    input_dir, turnover_dir = paths.get("data_analysis"), paths.get("turnover")

    df_main_metrics = pd.DataFrame()
    if not input_dir or not input_dir.exists():
        print(
            f"  - ⚠️ 警告：{period_label} 的 data_analysis_result 目录未找到: {input_dir}"
        )
    else:
        all_company_data = []
        metrics_to_extract = {
            k: v for k, v in POSTAL_METRICS_MAP.items() if k != "中转次数"
        }
        for company_key, file_prefix in COMPANY_FILE_MAP.items():
            file_path = input_dir / f"{file_prefix}_data_analysis_result.xlsx"
            if not file_path.exists():
                continue
            try:
                df_basic = pd.read_excel(file_path, sheet_name="基础指标").set_index(
                    "项目"
                )
                source_metrics = list(metrics_to_extract.values())
                if all(item in df_basic.index for item in source_metrics):
                    company_metrics = df_basic.loc[source_metrics, "mean"].rename(
                        company_key
                    )
                    all_company_data.append(company_metrics)
            except Exception as e:
                print(f"  - ⚠️ 警告：处理文件 {file_path.name} 失败: {e}")
        if all_company_data:
            df_period = pd.concat(all_company_data, axis=1).T
            df_period.rename(
                columns=lambda x: [k for k, v in metrics_to_extract.items() if v == x][
                    0
                ],
                inplace=True,
            )
            # (修改) 使用不含'快包'的列表进行行业对比
            industry_competitors = df_period.index.intersection(
                COMPANIES_FOR_INDUSTRY_COMPARISON
            ).tolist()
            if industry_competitors:
                df_period.loc["行业均值"] = df_period.loc[industry_competitors].mean()
                for metric in metrics_to_extract.keys():
                    if metric in HIGHER_IS_BETTER:
                        df_period.loc["行业最优", metric] = df_period.loc[
                            industry_competitors, metric
                        ].max()
                    else:
                        df_period.loc["行业最优", metric] = df_period.loc[
                            industry_competitors, metric
                        ].min()
            df_main_metrics = df_period.T

    df_turnover_metrics = get_turnover_data(turnover_dir)
    df_final = pd.concat([df_main_metrics, df_turnover_metrics])
    print(f"  ✅ {period_label} 数据提取完成。")
    return df_final


# ==============================================================================
# Part 2: 报告生成函数
# ==============================================================================
def create_postal_report_table(df_current, df_mom, df_yoy):
    report_cols = [
        "指标",
        "类别",
        "EMS",
        "邮政",
        "行业均值",
        "行业最优",
        "EMS排名",
        "邮政快包排名",
    ]
    source_value_cols = ["EMS", "快包", "行业均值", "行业最优"]
    report_value_cols = ["EMS", "邮政", "行业均值", "行业最优"]
    # 排名池已正确，无需修改
    ems_competitors = [c for c in COMPANIES_ALL if c != "快包"]
    postal_competitors = [c for c in COMPANIES_ALL if c != "EMS"]
    if not df_mom.empty:
        df_mom = df_mom.reindex(columns=df_current.columns)
    if not df_yoy.empty:
        df_yoy = df_yoy.reindex(columns=df_current.columns)
    report_rows = [
        f"{metric}_{p}"
        for metric in POSTAL_METRICS_MAP
        for p in ["本期", "环比", "同比"]
    ]
    table_df = pd.DataFrame(index=report_rows, columns=report_cols).fillna("")

    for metric in POSTAL_METRICS_MAP.keys():
        table_df.loc[f"{metric}_本期", "指标"] = metric.replace("（小时）", "")
        table_df.loc[f"{metric}_本期", "类别"] = CURRENT_MONTH_DISPLAY
        table_df.loc[f"{metric}_环比", "类别"] = "环比"
        table_df.loc[f"{metric}_同比", "类别"] = "同比"
        if not df_current.empty and metric in df_current.index:
            table_df.loc[f"{metric}_本期", report_value_cols] = df_current.loc[
                metric, source_value_cols
            ].values
        if not df_mom.empty and metric in df_mom.index:
            delta = (
                df_current.loc[metric, source_value_cols]
                - df_mom.loc[metric, source_value_cols]
            )
            table_df.loc[f"{metric}_环比", report_value_cols] = delta.round(4).values
        if not df_yoy.empty and metric in df_yoy.index:
            delta = (
                df_current.loc[metric, source_value_cols]
                - df_yoy.loc[metric, source_value_cols]
            )
            table_df.loc[f"{metric}_同比", report_value_cols] = delta.round(4).values

        ascending = metric not in HIGHER_IS_BETTER
        if not df_current.empty and metric in df_current.index:
            ranks = df_current.loc[metric, ems_competitors].rank(
                method="min", ascending=ascending, na_option="bottom"
            )
            table_df.loc[f"{metric}_本期", "EMS排名"] = ranks.get("EMS")
            ranks = df_current.loc[metric, postal_competitors].rank(
                method="min", ascending=ascending, na_option="bottom"
            )
            table_df.loc[f"{metric}_本期", "邮政快包排名"] = ranks.get("快包")
        if not df_mom.empty and metric in df_mom.index:
            ranks = df_mom.loc[metric, ems_competitors].rank(
                method="min", ascending=ascending, na_option="bottom"
            )
            table_df.loc[f"{metric}_环比", "EMS排名"] = ranks.get("EMS")
            ranks = df_mom.loc[metric, postal_competitors].rank(
                method="min", ascending=ascending, na_option="bottom"
            )
            table_df.loc[f"{metric}_环比", "邮政快包排名"] = ranks.get("快包")
        if not df_yoy.empty and metric in df_yoy.index:
            ranks = df_yoy.loc[metric, ems_competitors].rank(
                method="min", ascending=ascending, na_option="bottom"
            )
            table_df.loc[f"{metric}_同比", "EMS排名"] = ranks.get("EMS")
            ranks = df_yoy.loc[metric, postal_competitors].rank(
                method="min", ascending=ascending, na_option="bottom"
            )
            table_df.loc[f"{metric}_同比", "邮政快包排名"] = ranks.get("快包")

    table_df.rename(columns={"类别": "指标", "指标": "类别"}, inplace=True)
    return table_df[
        ["类别", "指标"]
        + [col for col in table_df.columns if col not in ["类别", "指标"]]
    ]


# ==============================================================================
# Part 3 & 最终执行入口
# ==============================================================================
def write_postal_formatted_excel(writer, df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    worksheet = writer.sheets[sheet_name]
    worksheet.set_column("A:H", 15)


if __name__ == "__main__":
    print(
        f"🚀 开始生成 {CURRENT_YEAR_MONTH} 月度邮政报告 表1：全程时限、72小时妥投率及中转次数排名..."
    )

    df_current_postal = get_postal_period_data("本期")
    df_mom_postal = get_postal_period_data("上月")
    df_yoy_postal = get_postal_period_data("去年同期")

    if df_current_postal.empty:
        print("🛑 本期邮政数据未能成功提取，无法生成报告。")
    else:
        postal_table_df = create_postal_report_table(
            df_current_postal, df_mom_postal, df_yoy_postal
        )
        output_path = (
            POSTAL_OUTPUT_DIR
            / f"1_全程时限72小时妥投率及中转次数排名_{CURRENT_YEAR_MONTH}.xlsx"
        )
        with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
            write_postal_formatted_excel(writer, postal_table_df, "时限及排名")
        print(f"\n🎉 邮政报告已成功保存至: {output_path}")

🚀 开始生成 202506 月度邮政报告 表1：全程时限、72小时妥投率及中转次数排名...

--- 正在处理邮政报告的 本期 数据 ---
  ✅ 本期 数据提取完成。

--- 正在处理邮政报告的 上月 数据 ---
  ✅ 上月 数据提取完成。

--- 正在处理邮政报告的 去年同期 数据 ---
  ✅ 去年同期 数据提取完成。

🎉 邮政报告已成功保存至: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输出/3_邮政报告表格/1_全程时限72小时妥投率及中转次数排名_202506.xlsx


In [7]:
# ==============================================================================
# Cell 3.2: 邮政报告所需表格生成 表2 分公里妥投率和中转次数
# ==============================================================================
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# --- 1. 全局配置 ---
ROOT_PATH = Path.cwd()
POSTAL_OUTPUT_DIR = ROOT_PATH / "报告数据" / "输出" / "3_邮政报告表格"
POSTAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def get_file_paths(period_label):
    base_dir = ROOT_PATH
    month_str_map = {"上月": PREVIOUS_YEAR_MONTH, "去年同期": LAST_YEAR_MONTH}
    if period_label != "本期":
        month_str = month_str_map.get(period_label, "")
        base_dir = base_dir / "报告数据" / "输入" / "historical_data" / month_str
        return {
            "data_analysis": base_dir / f"data_analysis_result_{month_str}",
            "turnover": base_dir / f"中转数据_{month_str}",
        }
    return {
        "data_analysis": base_dir / "报告数据" / "输出" / "data_analysis_result",
        "turnover": base_dir / "报告数据" / "temp" / "5_中转数据",
    }


# 明确定义各个公司列表的用途
COMPANIES_ALL = [
    "中通",
    "圆通",
    "极兔",
    "申通",
    "韵达",
    "顺丰",
    "京东",
    "EMS",
    "德邦",
    "快包",
]
# 用于行业对比的公司列表，明确排除'快包'
COMPANIES_FOR_INDUSTRY_COMPARISON = [c for c in COMPANIES_ALL if c != "快包"]
COMPANY_FILE_MAP = {"EMS": "EMS", "快包": "邮政"}


# --- 2. 核心计算函数 ---
def calculate_all_segmented_metrics_optimized(period_label):
    print(f"\n--- 正在计算 {period_label} 的所有分段指标 (最终版) ---")
    paths = get_file_paths(period_label)
    data_analysis_dir, turnover_dir = paths.get("data_analysis"), paths.get("turnover")

    if not data_analysis_dir or not data_analysis_dir.exists():
        print(f"  - ❌ 错误：数据源目录不存在: {data_analysis_dir}")
        return pd.DataFrame()

    all_company_results = {}

    for company_key in COMPANIES_ALL:
        file_prefix = COMPANY_FILE_MAP.get(company_key, company_key)
        detail_file = data_analysis_dir / f"{file_prefix}_data_analysis_result.xlsx"

        if not detail_file.exists():
            continue

        print(f"  - 正在处理: {company_key}")
        try:
            df_detail = pd.read_excel(detail_file, sheet_name="线路详细数据")
            df_summary = pd.read_excel(detail_file, sheet_name="线路汇总数据")
            if df_detail.empty or df_summary.empty:
                continue
        except Exception as e:
            print(f"    -> 警告: 读取文件 {detail_file.name} 失败: {e}")
            continue

        if "寄出城市" in df_detail.columns and "寄达城市" in df_detail.columns:
            df_detail["路线"] = (
                df_detail["寄出城市"].astype(str)
                + "-"
                + df_detail["寄达城市"].astype(str)
            )
        else:
            print(
                f"    -> 警告: {detail_file.name} (详细数据) 缺少'寄出城市'或'寄达城市'。"
            )
            continue

        if "路线" in df_summary.columns and "快递数量" in df_summary.columns:
            df = pd.merge(
                df_detail, df_summary[["路线", "快递数量"]], on="路线", how="left"
            )
        else:
            print(
                f"    -> 警告: {detail_file.name} (汇总数据) 缺少'路线'或'快递数量'，将使用算术平均。"
            )
            df = df_detail.copy()
            df["快递数量"] = 1

        turnover_file = (
            next(turnover_dir.glob(f"{file_prefix}*.xlsx"), None)
            if turnover_dir and turnover_dir.exists()
            else None
        )
        if turnover_file:
            try:
                df_turnover = pd.read_excel(turnover_file)
                if (
                    "出发城市" in df_turnover.columns
                    and "到达城市" in df_turnover.columns
                    and "平均中转次数" in df_turnover.columns
                ):
                    df_turnover["路线"] = (
                        df_turnover["出发城市"].astype(str)
                        + "-"
                        + df_turnover["到达城市"].astype(str)
                    )
                    df_turnover_agg = (
                        df_turnover.groupby("路线")[["平均中转次数"]]
                        .mean()
                        .reset_index()
                    )
                    df = pd.merge(df, df_turnover_agg, on="路线", how="left")
            except Exception as e:
                print(f"    -> 警告: 处理中转文件 {turnover_file.name} 时发生错误: {e}")

        required_cols = ["公里", "T+1_achieved", "T+2_achieved", "is_air"]
        if not all(c in df.columns for c in required_cols):
            print(f"    -> 警告: 文件 {detail_file.name} 缺少必要的预计算列，跳过。")
            continue

        df_under_600, df_over_600 = df[df["公里"] <= 600], df[df["公里"] > 600]
        df_over_600_air = df_over_600[df_over_600["is_air"] == True]

        if company_key in ["EMS", "快包"]:
            denominator = len(df_over_600_air)
            numerator = (
                df_over_600_air["T+1_achieved"].sum()
                if "T+1_achieved" in df_over_600_air
                else 0
            )
            print(
                f"    -> [审查] {company_key} 600km+航空件T+1妥投率: {numerator} / {denominator}"
            )

        def weighted_avg_turnover(df_segment):
            if (
                df_segment.empty
                or "平均中转次数" not in df_segment.columns
                or "快递数量" not in df_segment.columns
                or df_segment["平均中转次数"].isna().all()
            ):
                return np.nan
            valid_segment = df_segment.dropna(subset=["平均中转次数", "快递数量"])
            if valid_segment.empty or valid_segment["快递数量"].sum() == 0:
                return np.nan
            return np.average(
                valid_segment["平均中转次数"], weights=valid_segment["快递数量"]
            )

        all_company_results[company_key] = {
            "600公里以下 T+1妥投率": df_under_600["T+1_achieved"].mean(),
            "600公里以下 中转次数": weighted_avg_turnover(df_under_600),
            "600公里以上 全产品T+1妥投率": df_over_600["T+1_achieved"].mean(),
            "600公里以上 全产品T+2妥投率": df_over_600["T+2_achieved"].mean(),
            "600公里以上 全产品中转次数": weighted_avg_turnover(df_over_600),
            "600公里以上 航空件T+1妥投率": df_over_600_air["T+1_achieved"].mean()
            if not df_over_600_air.empty
            else np.nan,
        }

    if not all_company_results:
        return pd.DataFrame()
    df_results = pd.DataFrame.from_dict(all_company_results, orient="index")

    industry_metrics = list(df_results.columns)
    # (修改) 使用不含'快包'的列表进行行业对比
    industry_competitors = df_results.index.intersection(
        COMPANIES_FOR_INDUSTRY_COMPARISON
    ).tolist()
    if industry_competitors:
        df_results.loc["行业均值"] = df_results.loc[
            industry_competitors, industry_metrics
        ].mean()
        for metric in industry_metrics:
            if "妥投率" in metric:
                df_results.loc["行业最优", metric] = df_results.loc[
                    industry_competitors, metric
                ].max()
            else:
                df_results.loc["行业最优", metric] = df_results.loc[
                    industry_competitors, metric
                ].min()

    return df_results.loc[
        df_results.index.intersection(["EMS", "快包", "行业均值", "行业最优"])
    ].T


def format_to_report_style(df_period):
    if df_period.empty:
        return pd.DataFrame()
    df_final = (
        df_period.rename(columns={"快包": "邮政快包"})
        .reset_index()
        .rename(columns={"index": "指标"})
    )
    split_data = df_final["指标"].str.split(" ", n=1, expand=True)
    df_final.insert(0, "类别", split_data[0])
    df_final["指标"] = split_data[1]
    return df_final[["类别", "指标", "EMS", "邮政快包", "行业均值", "行业最优"]]


# --- 3. 主执行入口 ---
if __name__ == "__main__":
    print("🚀 开始生成 表2 分公里妥投率和中转次数...")
    df_current = calculate_all_segmented_metrics_optimized("本期")
    df_mom = calculate_all_segmented_metrics_optimized("上月")
    table_current = format_to_report_style(df_current)
    table_mom = format_to_report_style(df_mom)
    output_path = (
        POSTAL_OUTPUT_DIR / f"2_分公里妥投率和中转次数_{CURRENT_YEAR_MONTH}.xlsx"
    )
    with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
        if not table_current.empty:
            table_current.to_excel(writer, sheet_name="本期", index=False)
        if not table_mom.empty:
            table_mom.to_excel(writer, sheet_name="环比", index=False)
        workbook = writer.book
        percent_format, float_format = (
            workbook.add_format({"num_format": "0.00%"}),
            workbook.add_format({"num_format": "0.00"}),
        )
        for sheet_name in writer.sheets:
            worksheet, df_to_format = (
                writer.sheets[sheet_name],
                table_current if sheet_name == "本期" else table_mom,
            )
            if df_to_format.empty:
                continue
            for r in range(len(df_to_format)):
                cell_format = (
                    percent_format
                    if "妥投率" in str(df_to_format.iat[r, 1])
                    else float_format
                )
                for c in range(2, 6):
                    if pd.notna(df_to_format.iat[r, c]):
                        worksheet.write_number(
                            r + 1, c, df_to_format.iat[r, c], cell_format
                        )
            worksheet.set_column("A:B", 25)
            worksheet.set_column("C:F", 15)
    print(f"\n🎉 邮政月报 表2 分公里妥投率和中转次数 已成功保存至: {output_path}")

🚀 开始生成 表2 分公里妥投率和中转次数...

--- 正在计算 本期 的所有分段指标 (最终版) ---
  - 正在处理: 中通
  - 正在处理: 圆通
  - 正在处理: 极兔
  - 正在处理: 申通
  - 正在处理: 韵达
  - 正在处理: 顺丰
  - 正在处理: 京东
  - 正在处理: EMS
    -> [审查] EMS 600km+航空件T+1妥投率: 10007 / 24993
  - 正在处理: 德邦
  - 正在处理: 快包
    -> [审查] 快包 600km+航空件T+1妥投率: 111 / 1116

--- 正在计算 上月 的所有分段指标 (最终版) ---
  - 正在处理: 中通
  - 正在处理: 圆通
  - 正在处理: 极兔
  - 正在处理: 申通
  - 正在处理: 韵达
  - 正在处理: 顺丰
  - 正在处理: 京东
  - 正在处理: EMS
    -> [审查] EMS 600km+航空件T+1妥投率: 11996 / 15349
  - 正在处理: 德邦
  - 正在处理: 快包
    -> [审查] 快包 600km+航空件T+1妥投率: 123 / 1028

🎉 邮政月报 表2 分公里妥投率和中转次数 已成功保存至: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输出/3_邮政报告表格/2_分公里妥投率和中转次数_202506.xlsx


In [8]:
# ==============================================================================
# Cell 3.3: 邮政报告所需表格生成 - 表3
# ==============================================================================
# 确保在脚本开头有必要的导入
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter

# --- 1. 全局配置 ---
try:
    ROOT_PATH = Path.cwd()
    POSTAL_OUTPUT_DIR = ROOT_PATH / "报告数据" / "输出" / "3_邮政报告表格"
    POSTAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
except NameError:
    ROOT_PATH = Path(".")
    POSTAL_OUTPUT_DIR = ROOT_PATH / "报告数据" / "输出" / "3_邮政报告表格"
    POSTAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def get_file_paths(period_label):
    if period_label == "本期":
        return {
            "data_analysis": ROOT_PATH / "报告数据" / "输出" / "data_analysis_result"
        }
    elif period_label == "上月":
        return {
            "data_analysis": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / f"{PREVIOUS_YEAR_MONTH}"
            / f"data_analysis_result_{PREVIOUS_YEAR_MONTH}"
        }
    elif period_label == "去年同期":
        return {
            "data_analysis": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / f"{LAST_YEAR_MONTH}"
            / f"data_analysis_result_{LAST_YEAR_MONTH}"
        }
    return {}


COMPANY_FILE_MAP = {
    "EMS": "EMS",
    "快包": "邮政",
    "中通": "中通",
    "圆通": "圆通",
    "极兔": "极兔",
    "申通": "申通",
    "韵达": "韵达",
    "顺丰": "顺丰",
    "京东": "京东",
    "德邦": "德邦",
}
# 用于行业对比的公司列表，明确排除'快包'
COMPANIES_FOR_INDUSTRY_COMPARISON = [c for c in COMPANY_FILE_MAP.keys() if c != "快包"]

METRICS_OF_INTEREST = ["寄出地处理时限", "运输时限", "寄达地处理时限", "投递时限"]
CITY_SHEET_METRICS = ["寄出地处理时限", "寄达地处理时限", "投递时限"]
COMPANY_COL, RANK_SUFFIX, SHEET_NAME = "企业名称", "-排名", "线路详细数据"


# --- 2. 辅助函数 ---
def read_data_from_folder(folder_path, sheet_name=SHEET_NAME, company_col=COMPANY_COL):
    if not folder_path or not folder_path.is_dir():
        return pd.DataFrame()
    files_to_read = list(folder_path.glob("*.xlsx"))
    if not files_to_read:
        return pd.DataFrame()
    df_list = []
    for f_path in files_to_read:
        company_name_standard = next(
            (
                std_name
                for std_name, prefix in COMPANY_FILE_MAP.items()
                if prefix in f_path.name
            ),
            None,
        )
        if not company_name_standard:
            continue
        try:
            df = pd.read_excel(
                f_path,
                sheet_name=sheet_name,
                usecols=["寄出城市", "寄达城市"] + METRICS_OF_INTEREST,
            )
            df[company_col] = company_name_standard
            df_list.append(df)
        except Exception:
            continue
    if not df_list:
        return pd.DataFrame()
    return pd.concat(df_list, ignore_index=True)


def calculate_special_ranks(df):
    if df.empty:
        return df
    for metric in METRICS_OF_INTEREST:
        rank_col = f"{metric}{RANK_SUFFIX}"
        df[rank_col] = np.nan
        if "EMS" in df.index:
            df_for_ems = df.drop(index="快包", errors="ignore")
            df.loc["EMS", rank_col] = (
                df_for_ems[metric].rank(method="min", ascending=True).get("EMS")
            )
        if "快包" in df.index:
            df_for_kuai = df.drop(index="EMS", errors="ignore")
            df.loc["快包", rank_col] = (
                df_for_kuai[metric].rank(method="min", ascending=True).get("快包")
            )
    return df


# --- 3. 生成城市维度对比表的函数 ---
def create_city_comparison_sheet(
    company_name: str, df_current_raw, df_mom_raw, df_yoy_raw
):
    print(f"  - 正在为 '{company_name}' 生成城市维度数据...")
    origin_metric, dest_metrics = ["寄出地处理时限"], ["寄达地处理时限", "投递时限"]

    def get_period_data(df_raw, company):
        if df_raw.empty:
            return pd.DataFrame()
        df_company = df_raw[df_raw[COMPANY_COL] == company].copy()
        if df_company.empty:
            return pd.DataFrame()
        df_origin = df_company.groupby("寄出城市")[origin_metric].mean()
        df_dest = df_company.groupby("寄达城市")[dest_metrics].mean()
        df_merged = pd.merge(
            df_origin, df_dest, left_index=True, right_index=True, how="outer"
        )
        df_merged.index.name = "城市"
        return df_merged

    df_current_city = get_period_data(df_current_raw, company_name)
    df_mom_city = get_period_data(df_mom_raw, company_name)
    df_yoy_city = get_period_data(df_yoy_raw, company_name)

    df_current_city = df_current_city.add_suffix(f"_{CURRENT_YEAR_MONTH}")
    df_mom_city = df_mom_city.add_suffix(f"_{PREVIOUS_YEAR_MONTH}")
    df_yoy_city = df_yoy_city.add_suffix(f"_{LAST_YEAR_MONTH}")

    df_merged_all_periods = pd.concat(
        [df_current_city, df_mom_city, df_yoy_city], axis=1
    )
    final_cols_ordered = []

    for metric in CITY_SHEET_METRICS:
        current_col, mom_col, yoy_col = (
            f"{metric}_{CURRENT_YEAR_MONTH}",
            f"{metric}_{PREVIOUS_YEAR_MONTH}",
            f"{metric}_{LAST_YEAR_MONTH}",
        )
        df_merged_all_periods[f"{metric}_环比延长"] = df_merged_all_periods.get(
            current_col
        ) - df_merged_all_periods.get(mom_col)
        df_merged_all_periods[f"{metric}_同比延长"] = df_merged_all_periods.get(
            current_col
        ) - df_merged_all_periods.get(yoy_col)
        df_merged_all_periods[f"{metric}_环比延长幅度"] = np.where(
            df_merged_all_periods[mom_col].notna()
            & (df_merged_all_periods[mom_col] != 0),
            df_merged_all_periods[f"{metric}_环比延长"]
            / df_merged_all_periods[mom_col],
            np.nan,
        )
        df_merged_all_periods[f"{metric}_同比延长幅度"] = np.where(
            df_merged_all_periods[yoy_col].notna()
            & (df_merged_all_periods[yoy_col] != 0),
            df_merged_all_periods[f"{metric}_同比延长"]
            / df_merged_all_periods[yoy_col],
            np.nan,
        )
        final_cols_ordered.extend(
            [
                current_col,
                mom_col,
                yoy_col,
                f"{metric}_环比延长",
                f"{metric}_环比延长幅度",
                f"{metric}_同比延长",
                f"{metric}_同比延长幅度",
            ]
        )

    df_final = df_merged_all_periods.reindex(columns=final_cols_ordered).copy()
    return df_final.reset_index()


# --- 4. 核心执行函数 ---
def generate_segment_times_table():
    print("🚀 开始生成 表3 分环节时限及排名...")
    print("  - 正在读取当月、环比和同比的源数据...")
    df_current_raw = read_data_from_folder(get_file_paths("本期").get("data_analysis"))
    df_mom_raw = read_data_from_folder(get_file_paths("上月").get("data_analysis"))
    df_yoy_raw = read_data_from_folder(get_file_paths("去年同期").get("data_analysis"))
    if df_current_raw.empty:
        print(f"  - ❌ 错误：未找到当月数据，无法生成表格。")
        return

    print("  - [Sheet 1/3] 正在生成 '分环节时限分析'...")

    def aggregate_by_company(df_raw):
        return (
            df_raw.groupby(COMPANY_COL)[METRICS_OF_INTEREST].mean()
            if not df_raw.empty
            else pd.DataFrame()
        )

    df_current, df_mom_source, df_yoy_source = (
        aggregate_by_company(df_current_raw),
        aggregate_by_company(df_mom_raw),
        aggregate_by_company(df_yoy_raw),
    )
    df_current, df_mom_source, df_yoy_source = (
        calculate_special_ranks(df_current),
        calculate_special_ranks(df_mom_source),
        calculate_special_ranks(df_yoy_source),
    )

    # 使用不含'快包'的列表进行行业对比
    df_current_industry = df_current.loc[
        df_current.index.intersection(COMPANIES_FOR_INDUSTRY_COMPARISON)
    ]
    industry_mean_current = df_current_industry[METRICS_OF_INTEREST].mean()
    industry_best_current = df_current_industry[METRICS_OF_INTEREST].min()
    if not df_mom_source.empty:
        df_mom_industry = df_mom_source.loc[
            df_mom_source.index.intersection(COMPANIES_FOR_INDUSTRY_COMPARISON)
        ]
        industry_mean_mom, industry_best_mom = (
            df_mom_industry[METRICS_OF_INTEREST].mean(),
            df_mom_industry[METRICS_OF_INTEREST].min(),
        )
    else:
        industry_mean_mom, industry_best_mom = (
            pd.Series(dtype=float),
            pd.Series(dtype=float),
        )
    if not df_yoy_source.empty:
        df_yoy_industry = df_yoy_source.loc[
            df_yoy_source.index.intersection(COMPANIES_FOR_INDUSTRY_COMPARISON)
        ]
        industry_mean_yoy, industry_best_yoy = (
            df_yoy_industry[METRICS_OF_INTEREST].mean(),
            df_yoy_industry[METRICS_OF_INTEREST].min(),
        )
    else:
        industry_mean_yoy, industry_best_yoy = (
            pd.Series(dtype=float),
            pd.Series(dtype=float),
        )

    index_tuples = [
        (metric, p)
        for metric in METRICS_OF_INTEREST
        for p in [CURRENT_MONTH_DISPLAY, "环比", "同比"]
    ]
    multi_index = pd.MultiIndex.from_tuples(index_tuples, names=["指标", ""])
    result_columns = [
        "EMS",
        "邮政快包",
        "行业均值",
        "行业最优",
        "EMS排名",
        "邮政快包排名",
    ]
    df_sheet1 = pd.DataFrame(index=multi_index, columns=result_columns).sort_index()

    for metric in METRICS_OF_INTEREST:
        rank_col = f"{metric}{RANK_SUFFIX}"
        df_sheet1.loc[(metric, CURRENT_MONTH_DISPLAY), "EMS"] = (
            df_current.loc["EMS", metric] if "EMS" in df_current.index else np.nan
        )
        df_sheet1.loc[(metric, CURRENT_MONTH_DISPLAY), "邮政快包"] = (
            df_current.loc["快包", metric] if "快包" in df_current.index else np.nan
        )
        df_sheet1.loc[(metric, CURRENT_MONTH_DISPLAY), "行业均值"] = (
            industry_mean_current.get(metric)
        )
        df_sheet1.loc[(metric, CURRENT_MONTH_DISPLAY), "行业最优"] = (
            industry_best_current.get(metric)
        )
        df_sheet1.loc[(metric, CURRENT_MONTH_DISPLAY), "EMS排名"] = (
            df_current.loc["EMS", rank_col] if "EMS" in df_current.index else np.nan
        )
        df_sheet1.loc[(metric, CURRENT_MONTH_DISPLAY), "邮政快包排名"] = (
            df_current.loc["快包", rank_col] if "快包" in df_current.index else np.nan
        )
        if not df_mom_source.empty:
            current_ems_val = (
                df_current.loc["EMS", metric] if "EMS" in df_current.index else np.nan
            )
            mom_ems_val = (
                df_mom_source.loc["EMS", metric]
                if "EMS" in df_mom_source.index
                else np.nan
            )
            df_sheet1.loc[(metric, "环比"), "EMS"] = current_ems_val - mom_ems_val
            current_kuaibao_val = (
                df_current.loc["快包", metric] if "快包" in df_current.index else np.nan
            )
            mom_kuaibao_val = (
                df_mom_source.loc["快包", metric]
                if "快包" in df_mom_source.index
                else np.nan
            )
            df_sheet1.loc[(metric, "环比"), "邮政快包"] = (
                current_kuaibao_val - mom_kuaibao_val
            )
            df_sheet1.loc[(metric, "环比"), "行业均值"] = industry_mean_current.get(
                metric
            ) - industry_mean_mom.get(metric)
            df_sheet1.loc[(metric, "环比"), "行业最优"] = industry_best_current.get(
                metric
            ) - industry_best_mom.get(metric)
            df_sheet1.loc[(metric, "环比"), "EMS排名"] = (
                df_mom_source.loc["EMS", rank_col]
                if "EMS" in df_mom_source.index
                else np.nan
            )
            df_sheet1.loc[(metric, "环比"), "邮政快包排名"] = (
                df_mom_source.loc["快包", rank_col]
                if "快包" in df_mom_source.index
                else np.nan
            )
        if not df_yoy_source.empty:
            yoy_ems_val = (
                df_yoy_source.loc["EMS", metric]
                if "EMS" in df_yoy_source.index
                else np.nan
            )
            df_sheet1.loc[(metric, "同比"), "EMS"] = current_ems_val - yoy_ems_val
            yoy_kuaibao_val = (
                df_yoy_source.loc["快包", metric]
                if "快包" in df_yoy_source.index
                else np.nan
            )
            df_sheet1.loc[(metric, "同比"), "邮政快包"] = (
                current_kuaibao_val - yoy_kuaibao_val
            )
            df_sheet1.loc[(metric, "同比"), "行业均值"] = industry_mean_current.get(
                metric
            ) - industry_mean_yoy.get(metric)
            df_sheet1.loc[(metric, "同比"), "行业最优"] = industry_best_current.get(
                metric
            ) - industry_best_yoy.get(metric)
            df_sheet1.loc[(metric, "同比"), "EMS排名"] = (
                df_yoy_source.loc["EMS", rank_col]
                if "EMS" in df_yoy_source.index
                else np.nan
            )
            df_sheet1.loc[(metric, "同比"), "邮政快包排名"] = (
                df_yoy_source.loc["快包", rank_col]
                if "快包" in df_yoy_source.index
                else np.nan
            )

    print("  - [Sheet 2/3] 正在生成 'EMS同环比'...")
    df_sheet2_ems = create_city_comparison_sheet(
        "EMS", df_current_raw, df_mom_raw, df_yoy_raw
    )
    print("  - [Sheet 3/3] 正在生成 '邮政快包同环比'...")
    df_sheet3_kb = create_city_comparison_sheet(
        "快包", df_current_raw, df_mom_raw, df_yoy_raw
    )

    print("  - 正在格式化表格并保存至同一个Excel文件...")
    output_file_path = (
        POSTAL_OUTPUT_DIR / f"3_分环节时限及排名_{CURRENT_YEAR_MONTH}.xlsx"
    )
    with pd.ExcelWriter(output_file_path, engine="xlsxwriter") as writer:
        df_sheet1.to_excel(writer, sheet_name="分环节时限分析", na_rep="-")
        df_sheet2_ems.to_excel(writer, sheet_name="EMS同环比", index=False, na_rep="-")
        df_sheet3_kb.to_excel(
            writer, sheet_name="邮政快包同环比", index=False, na_rep="-"
        )

        workbook = writer.book
        float_format, int_format, percent_format = (
            workbook.add_format({"num_format": "0.00"}),
            workbook.add_format({"num_format": "0"}),
            workbook.add_format({"num_format": "0.00%"}),
        )
        worksheet1 = writer.sheets["分环节时限分析"]
        worksheet1.set_column("C:F", 12, float_format)
        worksheet1.set_column("G:H", 12, int_format)
        worksheet1.set_column("A:B", 18)

        for sheet_name in ["EMS同环比", "邮政快包同环比"]:
            if sheet_name not in writer.sheets:
                continue
            worksheet = writer.sheets[sheet_name]
            worksheet.set_column("A:A", 15)
            df_to_format = df_sheet2_ems if "EMS" in sheet_name else df_sheet3_kb
            if df_to_format.empty:
                continue
            for i, col_name in enumerate(df_to_format.columns[1:], 1):
                if "幅度" in col_name:
                    worksheet.set_column(i, i, 18, percent_format)
                else:
                    worksheet.set_column(i, i, 18, float_format)
    print(f"\n🎉 报告已成功保存至: {output_file_path}")


# --- 5. 主执行入口 ---
if __name__ == "__main__":
    generate_segment_times_table()

🚀 开始生成 表3 分环节时限及排名...
  - 正在读取当月、环比和同比的源数据...
  - [Sheet 1/3] 正在生成 '分环节时限分析'...
  - [Sheet 2/3] 正在生成 'EMS同环比'...
  - 正在为 'EMS' 生成城市维度数据...
  - [Sheet 3/3] 正在生成 '邮政快包同环比'...
  - 正在为 '快包' 生成城市维度数据...
  - 正在格式化表格并保存至同一个Excel文件...

🎉 报告已成功保存至: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输出/3_邮政报告表格/3_分环节时限及排名_202506.xlsx


In [ ]:
# ==============================================================================
# Cell 3.4: 邮政报告所需表格生成 表4-7 分公里段时限及排名
# ==============================================================================
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# --- 1. 全局配置 ---
ROOT_PATH = Path.cwd()
POSTAL_OUTPUT_DIR = ROOT_PATH / "报告数据" / "输出" / "3_邮政报告表格"
POSTAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


def get_file_paths(period_label):
    if period_label == "本期":
        return {
            "data_analysis": ROOT_PATH / "报告数据" / "输出" / "data_analysis_result"
        }
    elif period_label == "上月":
        return {
            "data_analysis": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / f"{PREVIOUS_YEAR_MONTH}"
            / f"data_analysis_result_{PREVIOUS_YEAR_MONTH}"
        }
    elif period_label == "去年同期":
        return {
            "data_analysis": ROOT_PATH
            / "报告数据"
            / "输入"
            / "historical_data"
            / f"{LAST_YEAR_MONTH}"
            / f"data_analysis_result_{LAST_YEAR_MONTH}"
        }
    return {}


COMPANY_FILE_MAP = {
    "EMS": "EMS",
    "快包": "邮政",
    "中通": "中通",
    "圆通": "圆通",
    "极兔": "极兔",
    "申通": "申通",
    "韵达": "韵达",
    "顺丰": "顺丰",
    "京东": "京东",
    "德邦": "德邦",
}
# 用于行业对比的公司列表，明确排除'快包'
COMPANIES_FOR_INDUSTRY_COMPARISON = [c for c in COMPANY_FILE_MAP.keys() if c != "快包"]

METRICS_OF_INTEREST = [
    "全程时限",
    "寄出地处理时限",
    "运输时限",
    "寄达地处理时限",
    "投递时限",
]
RANK_SUFFIX, SHEET_NAME = "-排名", "基础指标"
DISTANCE_COLS = {
    "表4：600公里以下（含600公里）": {"col_name": "0-600", "label": "0-600"},
    "表5：600-1500公里（含1500公里）": {"col_name": "600-1500", "label": "600-1500"},
    "表6：1500-2500公里（含2500公里）": {"col_name": "1500-2500", "label": "1500-2500"},
    "表7：2500公里以上": {"col_name": "2500以上", "label": "2500以上"},
}


# --- 2. 辅助函数 ---
def extract_metrics_from_folder(folder_path, sheet_name=SHEET_NAME):
    if not folder_path or not folder_path.is_dir():
        return {}
    all_company_data = {}
    for f_path in folder_path.glob("*.xlsx"):
        company_name_standard = next(
            (
                std_name
                for std_name, prefix in COMPANY_FILE_MAP.items()
                if prefix in f_path.name
            ),
            None,
        )
        if not company_name_standard:
            continue
        try:
            df = pd.read_excel(f_path, sheet_name=sheet_name, index_col="项目")
            df.columns = df.columns.str.strip()
            all_company_data[company_name_standard] = df
        except Exception:
            continue
    return all_company_data


def calculate_special_ranks(series):
    ranks = {}
    if "EMS" in series.index:
        ranks["EMS"] = (
            series.drop(index="快包", errors="ignore")
            .rank(method="min", ascending=True)
            .get("EMS")
        )
    if "快包" in series.index:
        ranks["快包"] = (
            series.drop(index="EMS", errors="ignore")
            .rank(method="min", ascending=True)
            .get("快包")
        )
    return ranks


# --- 3. 核心执行函数 ---
def generate_distance_segmented_tables():
    print("🚀 开始生成 表4-7 分公里段时限及排名...")
    print("  - 正在提取所有周期的基础指标数据...")
    current_data = extract_metrics_from_folder(
        get_file_paths("本期").get("data_analysis")
    )
    mom_data = extract_metrics_from_folder(get_file_paths("上月").get("data_analysis"))
    yoy_data = extract_metrics_from_folder(
        get_file_paths("去年同期").get("data_analysis")
    )

    if not current_data:
        print(
            f"  - ❌ 错误：未找到当期数据 (来自 data_analysis_result), 无法生成表格。"
        )
        return

    output_file_path = (
        POSTAL_OUTPUT_DIR / f"4_分公里时限及排名_{CURRENT_YEAR_MONTH}.xlsx"
    )
    with pd.ExcelWriter(output_file_path, engine="xlsxwriter") as writer:
        print("\n" + "=" * 80)
        for sheet_title, segment in DISTANCE_COLS.items():
            dist_col = segment["col_name"]
            print(f"📊 正在处理公里段: {dist_col} (Sheet: {sheet_title})")

            all_metrics_current, all_metrics_mom, all_metrics_yoy = {}, {}, {}
            for metric in METRICS_OF_INTEREST:
                all_metrics_current[metric] = pd.Series(
                    {
                        comp: df.loc[metric, dist_col]
                        for comp, df in current_data.items()
                        if metric in df.index and dist_col in df.columns
                    }
                )
                all_metrics_mom[metric] = pd.Series(
                    {
                        comp: df.loc[metric, dist_col]
                        for comp, df in mom_data.items()
                        if metric in df.index and dist_col in df.columns
                    }
                )
                all_metrics_yoy[metric] = pd.Series(
                    {
                        comp: df.loc[metric, dist_col]
                        for comp, df in yoy_data.items()
                        if metric in df.index and dist_col in df.columns
                    }
                )

            index_tuples = [
                (metric, p)
                for metric in METRICS_OF_INTEREST
                for p in [CURRENT_MONTH_DISPLAY, "环比", "同比"]
            ]
            multi_index = pd.MultiIndex.from_tuples(
                index_tuples, names=[f"指标{segment['label']}", ""]
            )
            final_df = pd.DataFrame(
                index=multi_index,
                columns=[
                    "EMS",
                    "邮政快包",
                    "行业均值",
                    "行业最优",
                    "EMS排名",
                    "邮政快包排名",
                ],
            )

            for metric in METRICS_OF_INTEREST:
                series_current, series_mom, series_yoy = (
                    all_metrics_current.get(metric, pd.Series(dtype=float)),
                    all_metrics_mom.get(metric, pd.Series(dtype=float)),
                    all_metrics_yoy.get(metric, pd.Series(dtype=float)),
                )
                if series_current.empty:
                    continue

                ranks_current, ranks_mom, ranks_yoy = (
                    calculate_special_ranks(series_current),
                    calculate_special_ranks(series_mom),
                    calculate_special_ranks(series_yoy),
                )

                # 使用不含'快包'的列表进行行业对比
                series_current_industry = series_current.drop(
                    labels=["快包"], errors="ignore"
                )
                industry_mean_current, industry_best_current = (
                    series_current_industry.mean(),
                    series_current_industry.min(),
                )

                series_mom_industry = series_mom.drop(labels=["快包"], errors="ignore")
                industry_mean_mom, industry_best_mom = (
                    series_mom_industry.mean(),
                    series_mom_industry.min(),
                )

                series_yoy_industry = series_yoy.drop(labels=["快包"], errors="ignore")
                industry_mean_yoy, industry_best_yoy = (
                    series_yoy_industry.mean(),
                    series_yoy_industry.min(),
                )

                # 填充本期数据
                final_df.loc[(metric, CURRENT_MONTH_DISPLAY), "EMS"] = (
                    series_current.get("EMS")
                )
                final_df.loc[(metric, CURRENT_MONTH_DISPLAY), "邮政快包"] = (
                    series_current.get("快包")
                )
                final_df.loc[(metric, CURRENT_MONTH_DISPLAY), "行业均值"] = (
                    industry_mean_current
                )
                final_df.loc[(metric, CURRENT_MONTH_DISPLAY), "行业最优"] = (
                    industry_best_current
                )
                final_df.loc[(metric, CURRENT_MONTH_DISPLAY), "EMS排名"] = (
                    ranks_current.get("EMS")
                )
                final_df.loc[(metric, CURRENT_MONTH_DISPLAY), "邮政快包排名"] = (
                    ranks_current.get("快包")
                )

                # 填充环比数据
                final_df.loc[(metric, "环比"), "EMS"] = series_current.get(
                    "EMS", np.nan
                ) - series_mom.get("EMS", np.nan)
                final_df.loc[(metric, "环比"), "邮政快包"] = series_current.get(
                    "快包", np.nan
                ) - series_mom.get("快包", np.nan)
                final_df.loc[(metric, "环比"), "行业均值"] = (
                    industry_mean_current - industry_mean_mom
                )
                final_df.loc[(metric, "环比"), "行业最优"] = (
                    industry_best_current - industry_best_mom
                )
                final_df.loc[(metric, "环比"), "EMS排名"] = ranks_mom.get("EMS")
                final_df.loc[(metric, "环比"), "邮政快包排名"] = ranks_mom.get("快包")

                # 填充同比数据
                final_df.loc[(metric, "同比"), "EMS"] = series_current.get(
                    "EMS", np.nan
                ) - series_yoy.get("EMS", np.nan)
                final_df.loc[(metric, "同比"), "邮政快包"] = series_current.get(
                    "快包", np.nan
                ) - series_yoy.get("快包", np.nan)
                final_df.loc[(metric, "同比"), "行业均值"] = (
                    industry_mean_current - industry_mean_yoy
                )
                final_df.loc[(metric, "同比"), "行业最优"] = (
                    industry_best_current - industry_best_yoy
                )
                final_df.loc[(metric, "同比"), "EMS排名"] = ranks_yoy.get("EMS")
                final_df.loc[(metric, "同比"), "邮政快包排名"] = ranks_yoy.get("快包")

            final_df.to_excel(writer, sheet_name=sheet_title, na_rep="-")
            worksheet, workbook = writer.sheets[sheet_title], writer.book
            float_format, int_format = (
                workbook.add_format({"num_format": "0.00", "align": "center"}),
                workbook.add_format({"num_format": "0", "align": "center"}),
            )
            worksheet.set_column("C:F", 12, float_format)
            worksheet.set_column("G:H", 15, int_format)
            worksheet.set_column("A:B", 18)
            print(f"    - ✅ Sheet '{sheet_title}' 生成成功。")

    print("\n" + "=" * 80)
    print(f"\n🎉 邮政月报 表4-7 分公里段时限及排名 已成功保存至: {output_file_path}")


# --- 4. 主执行入口 ---
if __name__ == "__main__":
    # 假设报告周期变量已定义
    # CURRENT_YEAR_MONTH, PREVIOUS_YEAR_MONTH, LAST_YEAR_MONTH, CURRENT_MONTH_DISPLAY
    generate_distance_segmented_tables()

🚀 开始生成 表4-7 分公里段时限及排名...
  - 正在提取所有周期的基础指标数据...

📊 正在处理公里段: 0-600 (Sheet: 表4：600公里以下（含600公里）)
    - ✅ Sheet '表4：600公里以下（含600公里）' 生成成功。
📊 正在处理公里段: 600-1500 (Sheet: 表5：600-1500公里（含1500公里）)
    - ✅ Sheet '表5：600-1500公里（含1500公里）' 生成成功。
📊 正在处理公里段: 1500-2500 (Sheet: 表6：1500-2500公里（含2500公里）)
    - ✅ Sheet '表6：1500-2500公里（含2500公里）' 生成成功。
📊 正在处理公里段: 2500以上 (Sheet: 表7：2500公里以上)
    - ✅ Sheet '表7：2500公里以上' 生成成功。


🎉 邮政月报 表4-7 分公里段时限及排名 已成功保存至: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输出/3_邮政报告表格/4_分公里时限及排名_202506.xlsx


In [ ]:
# ==============================================================================
# Cell 3.5: 邮政报告所需表格生成 表8-11 三大经济圈城市互寄时限
# ==============================================================================
# 确保在脚本开头有必要的导入
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter

# --- 1. 全局配置 ---
try:
    ROOT_PATH = Path.cwd()
    POSTAL_OUTPUT_DIR = ROOT_PATH / "报告数据" / "输出" / "3_邮政报告表格"
    POSTAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
except NameError:
    ROOT_PATH = Path(".")
    POSTAL_OUTPUT_DIR = ROOT_PATH / "报告数据" / "输出" / "3_邮政报告表格"
    POSTAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

current_file_path = ROOT_PATH / "报告数据" / "输出" / "分析总报告.xlsx"
prev_file_path = (
    ROOT_PATH
    / "报告数据"
    / "输入"
    / "historical_data"
    / f"{PREVIOUS_YEAR_MONTH}"
    / f"分析总报告_{PREVIOUS_YEAR_MONTH}.xlsx"
)
last_year_file_path = (
    ROOT_PATH
    / "报告数据"
    / "输入"
    / "historical_data"
    / f"{LAST_YEAR_MONTH}"
    / f"分析总报告_{LAST_YEAR_MONTH}.xlsx"
)
DATA_SHEET_NAME = "最终线路明细结果"

ALL_COMPANIES = [
    "EMS",
    "邮政快包",
    "中通",
    "圆通",
    "极兔",
    "申通",
    "韵达",
    "顺丰",
    "京东",
    "德邦",
]
# 用于行业对比的公司列表，明确排除'邮政快包'
COMPANIES_FOR_INDUSTRY_COMPARISON = [c for c in ALL_COMPANIES if c != "邮政快包"]
METRICS_OF_INTEREST = [
    "全程时限",
    "寄出地处理时限",
    "运输时限",
    "寄达地处理时限",
    "投递时限",
]
COMPANY_FILE_MAP = {
    "邮政快包": "快包",
    **{c: c for c in ALL_COMPANIES if c != "邮政快包"},
}
NUMERIC_COLS, RANK_COLS = (
    ["EMS", "邮政快包", "行业均值", "行业最优"],
    ["EMS排名", "邮政快包排名"],
)
FINAL_COLS = NUMERIC_COLS + RANK_COLS


# =================================================================================
# --- 2. 核心数据处理函数 ---
# =================================================================================
def calculate_metrics_from_report(file_path, sheet_name):
    if not file_path.exists():
        print(f"!! 错误: 文件未找到: {file_path}")
        return pd.DataFrame()
    try:
        print(f"  - 正在从文件 '{file_path.name}' 的 sheet '{sheet_name}' 读取数据...")
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        df.columns = df.columns.str.strip()
    except Exception as e:
        print(f"!! 错误: 读取文件 '{file_path.name}' 的 sheet '{sheet_name}' 失败: {e}")
        return pd.DataFrame()
    if "城市圈" not in df.columns:
        print(f"!! 严重错误: 在文件 '{file_path.name}' 中找不到关键列 '城市圈'。")
        return pd.DataFrame()

    df["城市圈"] = df["城市圈"].astype(str).str.strip()
    results = []

    # 经济圈数据处理
    for circle in ["京津冀", "长三角", "珠三角"]:
        df_circle = df[df["城市圈"] == circle]
        for metric in METRICS_OF_INTEREST:
            metric_results = {"区域": circle, "指标": metric}
            company_values = {}
            if not df_circle.empty:
                for company_name, file_suffix in COMPANY_FILE_MAP.items():
                    qty_col, metric_col = (
                        f"快递数量{file_suffix}",
                        f"{metric}{file_suffix}",
                    )
                    if qty_col in df_circle.columns and metric_col in df_circle.columns:
                        subset = df_circle[[qty_col, metric_col]].copy()
                        subset[qty_col] = pd.to_numeric(
                            subset[qty_col], errors="coerce"
                        ).fillna(0)
                        subset[metric_col] = pd.to_numeric(
                            subset[metric_col], errors="coerce"
                        )
                        total_qty = subset[qty_col].sum()
                        if total_qty > 0:
                            company_values[company_name] = (
                                subset[metric_col].fillna(0) * subset[qty_col]
                            ).sum() / total_qty
            for company in ALL_COMPANIES:
                metric_results[company] = company_values.get(company, np.nan)
            valid_values = pd.Series(company_values).dropna()
            # (修改) 使用不含'快包'的列表进行行业对比
            industry_values = valid_values.drop(labels=["邮政快包"], errors="ignore")
            metric_results.update(
                {
                    "行业均值": industry_values.mean()
                    if not industry_values.empty
                    else np.nan,
                    "行业最优": industry_values.min()
                    if not industry_values.empty
                    else np.nan,
                    "EMS排名": valid_values.drop("邮政快包", errors="ignore")
                    .rank(method="min")
                    .get("EMS", np.nan)
                    if not valid_values.empty
                    else np.nan,
                    "邮政快包排名": valid_values.drop("EMS", errors="ignore")
                    .rank(method="min")
                    .get("邮政快包", np.nan)
                    if not valid_values.empty
                    else np.nan,
                }
            )
            results.append(metric_results)

    # 汇总数据处理
    summary_definitions = {
        "全国全程时限": {"data": df, "calc_metric": "全程时限"},
        "经济圈互寄全程时限": {
            "data": df[df["城市圈"].isin(["京津冀", "长三角", "珠三角"])],
            "calc_metric": "全程时限",
        },
    }
    for label_metric, config in summary_definitions.items():
        data_subset, calc_metric = config["data"], config["calc_metric"]
        metric_results = {"区域": "汇总", "指标": label_metric}
        company_values = {}
        if not data_subset.empty:
            for company_name, file_suffix in COMPANY_FILE_MAP.items():
                qty_col, metric_col = (
                    f"快递数量{file_suffix}",
                    f"{calc_metric}{file_suffix}",
                )
                if qty_col in data_subset.columns and metric_col in data_subset.columns:
                    subset = data_subset[[qty_col, metric_col]].copy()
                    subset[qty_col] = pd.to_numeric(
                        subset[qty_col], errors="coerce"
                    ).fillna(0)
                    total_qty = subset[qty_col].sum()
                    if total_qty > 0:
                        subset[metric_col] = pd.to_numeric(
                            subset[metric_col], errors="coerce"
                        )
                        company_values[company_name] = (
                            subset[metric_col].fillna(0) * subset[qty_col]
                        ).sum() / total_qty
        for company in ALL_COMPANIES:
            metric_results[company] = company_values.get(company, np.nan)
        valid_values = pd.Series(company_values).dropna()
        # (修改) 使用不含'快包'的列表进行行业对比
        industry_values = valid_values.drop(labels=["邮政快包"], errors="ignore")
        metric_results.update(
            {
                "行业均值": industry_values.mean()
                if not industry_values.empty
                else np.nan,
                "行业最优": industry_values.min()
                if not industry_values.empty
                else np.nan,
                "EMS排名": valid_values.drop("邮政快包", errors="ignore")
                .rank(method="min")
                .get("EMS", np.nan)
                if not valid_values.empty
                else np.nan,
                "邮政快包排名": valid_values.drop("EMS", errors="ignore")
                .rank(method="min")
                .get("邮政快包", np.nan)
                if not valid_values.empty
                else np.nan,
            }
        )
        results.append(metric_results)

    return pd.DataFrame(results)


def create_final_table(
    df_current, df_prev, df_last_year, region_name, metrics_list, first_col_name
):
    if df_current.empty:
        return pd.DataFrame()
    current_region = df_current[df_current["区域"] == region_name].set_index("指标")
    prev_region = (
        df_prev[df_prev["区域"] == region_name].set_index("指标")
        if not df_prev.empty
        else pd.DataFrame()
    )
    year_region = (
        df_last_year[df_last_year["区域"] == region_name].set_index("指标")
        if not df_last_year.empty
        else pd.DataFrame()
    )

    data_rows = []
    for metric in metrics_list:
        if metric not in current_region.index:
            continue
        current_series = current_region.loc[metric]
        prev_series = (
            prev_region.loc[metric]
            if not prev_region.empty and metric in prev_region.index
            else pd.Series(dtype=object)
        )
        year_series = (
            year_region.loc[metric]
            if not year_region.empty and metric in year_region.index
            else pd.Series(dtype=object)
        )
        row_current = current_series
        row_hb = pd.Series(index=FINAL_COLS, dtype=object)
        row_hb[NUMERIC_COLS] = current_series[NUMERIC_COLS] - prev_series[NUMERIC_COLS]
        row_hb[RANK_COLS] = prev_series[RANK_COLS]
        row_tb = pd.Series(index=FINAL_COLS, dtype=object)
        row_tb[NUMERIC_COLS] = current_series[NUMERIC_COLS] - year_series[NUMERIC_COLS]
        row_tb[RANK_COLS] = year_series[RANK_COLS]
        data_rows.extend([row_current, row_hb, row_tb])

    if not data_rows:
        return pd.DataFrame()
    df = pd.DataFrame(data_rows, columns=FINAL_COLS)
    index_tuples = []
    for metric in metrics_list:
        if metric in current_region.index:
            index_tuples.extend(
                [(metric, CURRENT_MONTH_DISPLAY), (metric, "环比"), (metric, "同比")]
            )
    df.index = pd.MultiIndex.from_tuples(index_tuples, names=[first_col_name, "指标"])
    return df


def format_excel_sheet(ws, num_format="0.00", rank_format="0"):
    if ws.max_row <= 1:
        return
    for row in ws.iter_rows(min_row=1):
        for cell in row:
            if cell.row == 1:
                cell.font, cell.alignment = (
                    Font(bold=True),
                    Alignment(horizontal="center", vertical="center"),
                )
                continue
            cell.alignment = Alignment(horizontal="center", vertical="center")
            if cell.column in [3, 4, 5, 6]:
                cell.number_format = num_format
            elif cell.column in [7, 8]:
                if pd.isna(cell.value):
                    cell.value = "-"
                else:
                    cell.number_format = rank_format
    start_row = 2
    for r in range(3, ws.max_row + 2):
        if r > ws.max_row or ws.cell(r, 1).value != ws.cell(start_row, 1).value:
            if start_row < r - 1:
                ws.merge_cells(
                    start_row=start_row, start_column=1, end_row=r - 1, end_column=1
                )
            ws.cell(start_row, 1).alignment = Alignment(
                horizontal="center", vertical="center", wrap_text=True
            )
            start_row = r
    ws.column_dimensions[ws.cell(row=1, column=1).column_letter].width = 20
    ws.column_dimensions[ws.cell(row=1, column=2).column_letter].width = 10
    for i in range(3, ws.max_column + 1):
        ws.column_dimensions[get_column_letter(i)].width = 14


# =================================================================================
# --- 3. 主执行流程 ---
# =================================================================================
if __name__ == "__main__":
    # 假设报告周期变量已定义
    # CURRENT_YEAR_MONTH, PREVIOUS_YEAR_MONTH, LAST_YEAR_MONTH, CURRENT_MONTH_DISPLAY
    print("--- 开始生成 5_三大经济圈城市互寄时限 报告 ---")
    print("\n[步骤1/4] 正在加载并处理各时期数据...")
    df_current = calculate_metrics_from_report(
        current_file_path, sheet_name=DATA_SHEET_NAME
    )
    df_prev = calculate_metrics_from_report(prev_file_path, sheet_name=DATA_SHEET_NAME)
    df_last_year = calculate_metrics_from_report(
        last_year_file_path, sheet_name=DATA_SHEET_NAME
    )
    print("\n[步骤1/4] 数据加载处理完成。")
    output_filename = POSTAL_OUTPUT_DIR / "5_三大经济圈城市互寄时限.xlsx"
    print(f"\n[步骤2/4] 正在生成各Sheet并写入到: {output_filename}")
    with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
        all_sheets = {
            "表8：京津冀城市互寄全程时限及分环节时限": (
                "京津冀",
                METRICS_OF_INTEREST,
                "京津冀城市互寄",
            ),
            "表9：长三角城市互寄全程时限及分环节时限": (
                "长三角",
                METRICS_OF_INTEREST,
                "长三角城市互寄",
            ),
            "表10：珠三角城市互寄全程时限及分环节时限": (
                "珠三角",
                METRICS_OF_INTEREST,
                "珠三角城市互寄",
            ),
            "表11：三大经济圈之间互寄全程时限": (
                "汇总",
                ["全国全程时限", "经济圈互寄全程时限"],
                "三大经济圈之间互寄",
            ),
        }
        for sheet_name, (region, metrics, col_name) in all_sheets.items():
            final_df = create_final_table(
                df_current, df_prev, df_last_year, region, metrics, col_name
            )
            final_df.to_excel(writer, sheet_name=sheet_name)
            print(f"  - Sheet '{sheet_name}' 已生成.")
    print("[步骤2/4] Excel基础文件写入完成。")

    print("\n[步骤3/4] 正在对Excel文件进行最终格式化...")
    wb = load_workbook(output_filename)
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        format_excel_sheet(ws)
        print(f"  - Sheet '{sheet_name}' 格式化完成.")
    wb.save(output_filename)
    print("[步骤4/4] 格式化完成并已保存。")
    print("\n" + "=" * 60)
    print(f"🎉 任务完成！报表已成功生成。")
    print(f"文件路径: {output_filename.resolve()}")
    print("=" * 60)

--- 开始生成 5_三大经济圈城市互寄时限 报告 ---

[步骤1/4] 正在加载并处理各时期数据...
  - 正在从文件 '分析总报告.xlsx' 的 sheet '最终线路明细结果' 读取数据...
  - 正在从文件 '分析总报告_202505.xlsx' 的 sheet '最终线路明细结果' 读取数据...
  - 正在从文件 '分析总报告_202406.xlsx' 的 sheet '最终线路明细结果' 读取数据...

[步骤1/4] 数据加载处理完成。

[步骤2/4] 正在生成各Sheet并写入到: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输出/3_邮政报告表格/5_三大经济圈城市互寄时限.xlsx
  - Sheet '表8：京津冀城市互寄全程时限及分环节时限' 已生成.
  - Sheet '表9：长三角城市互寄全程时限及分环节时限' 已生成.
  - Sheet '表10：珠三角城市互寄全程时限及分环节时限' 已生成.
  - Sheet '表11：三大经济圈之间互寄全程时限' 已生成.
[步骤2/4] Excel基础文件写入完成。

[步骤3/4] 正在对Excel文件进行最终格式化...
  - Sheet '表8：京津冀城市互寄全程时限及分环节时限' 格式化完成.
  - Sheet '表9：长三角城市互寄全程时限及分环节时限' 格式化完成.
  - Sheet '表10：珠三角城市互寄全程时限及分环节时限' 格式化完成.
  - Sheet '表11：三大经济圈之间互寄全程时限' 格式化完成.
[步骤4/4] 格式化完成并已保存。

🎉 任务完成！报表已成功生成。
文件路径: /Users/lava/Documents/国家邮政局发展研究中心实习/python_data_analysis/报告数据/输出/3_邮政报告表格/5_三大经济圈城市互寄时限.xlsx
